In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,938 kB]
Hit:13 htt

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-28 23:18:25--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.14MB/s    in 0.2s    

2022-08-28 23:18:25 (6.14 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Books_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...|2015-08-31 00:00:00|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend| 

In [5]:
from pyspark.sql.types import IntegerType
df = df.withColumn("star_rating", df["star_rating"].cast(IntegerType()))
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'timestamp')]

In [6]:
# Create the vine_table. DataFrame

vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes", "vine", "verified_purchase"]).drop_duplicates(["review_id"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R10003J44S385F|          5|            0|          0|   N|                Y|
|R10003SNMOFW8K|          4|            0|          0|   N|                N|
|R10008PS3BSEUD|          5|            0|          0|   N|                Y|
|R10009UDW9915R|          5|            2|          4|   N|                N|
|R1000AQ7RCGKUL|          5|            0|          1|   N|                Y|
|R1000I7I8TNAYW|          5|            0|          0|   N|                N|
|R1000KUSU1L66U|          5|            0|          0|   N|                Y|
|R1000MV3S5G9P8|          5|            0|          0|   N|                Y|
|R1000NIVBNK4FD|          4|            0|          0|   N|                Y|
|R1000Y5VKOPTHJ|          5|            0|          0|   N|     

In [7]:
# votes greater than 20
votes_df = vine_df.filter("total_votes>=20")
votes_df.show() 

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1001GN6Q1AF8U|          5|           49|         52|   N|                N|
|R1002YLZ8YP585|          1|            6|         20|   N|                Y|
|R1008VVANR2IBK|          1|            9|         29|   N|                N|
|R100HEX46R0ERT|          4|            3|         21|   N|                Y|
|R100IFSAD2WKOO|          5|           48|         51|   Y|                N|
| R100J3H0Y9QEM|          5|           21|         22|   N|                Y|
|R100JHUOQSRZ2H|          4|          129|        131|   N|                Y|
|R100LYR8JPWSTF|          1|            3|         34|   N|                Y|
|R100P15L69LT8Y|          5|           17|         20|   N|                Y|
|R101JKGO8DJVK8|          5|           30|         31|   N|     

In [8]:
# votes where helpful votes are at least 50% of total votes
helpful_df = votes_df.filter("helpful_votes/total_votes >= 0.5")
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1001GN6Q1AF8U|          5|           49|         52|   N|                N|
|R100IFSAD2WKOO|          5|           48|         51|   Y|                N|
| R100J3H0Y9QEM|          5|           21|         22|   N|                Y|
|R100JHUOQSRZ2H|          4|          129|        131|   N|                Y|
|R100P15L69LT8Y|          5|           17|         20|   N|                Y|
|R101JKGO8DJVK8|          5|           30|         31|   N|                Y|
|R101MO7FBZP8OV|          4|           31|         31|   N|                Y|
|R101PYWOBNRA12|          5|           22|         22|   N|                Y|
|R1027NH9ETHTVA|          4|           49|         52|   N|                Y|
|R102X6PTY1J5FB|          5|           24|         30|   N|     

In [9]:
# paid reviews where vine is yes
paid_review_df = helpful_df.filter(helpful_df["vine"]== "Y")
paid_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100IFSAD2WKOO|          5|           48|         51|   Y|                N|
|R10LRJXK046Y34|          5|           41|         42|   Y|                N|
|R10NK6QYS1BOOG|          4|           69|         74|   Y|                N|
|R10OE71JMLNDD5|          1|          360|        395|   Y|                N|
|R10OJRVVS7DKRQ|          3|           25|         26|   Y|                N|
|R10TGEU44BVNXW|          5|           32|         34|   Y|                N|
|R10VGA3Z0VP7F8|          5|           15|         21|   Y|                N|
|R10ZB1QOFTW3WR|          2|           61|         98|   Y|                N|
|R110B2P40TYCOH|          5|           26|         31|   Y|                N|
|R112C6RN5UAN3D|          4|           73|         76|   Y|     

In [10]:
# total paid reviews
paid_review_df.count()

5012

In [11]:
# total paid 5 star reviews
paid_review_df.filter("star_rating ==5").count()

2031

In [12]:
# percentage of paid 5 star reviews
paid_review_df.filter("star_rating ==5").count()/paid_review_df.count()*100

40.52274541101357

In [13]:
# unpaid reviews where vine is no
unpaid_review_df = helpful_df.filter(helpful_df["vine"]== "N")
unpaid_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1001GN6Q1AF8U|          5|           49|         52|   N|                N|
| R100J3H0Y9QEM|          5|           21|         22|   N|                Y|
|R100JHUOQSRZ2H|          4|          129|        131|   N|                Y|
|R100P15L69LT8Y|          5|           17|         20|   N|                Y|
|R101JKGO8DJVK8|          5|           30|         31|   N|                Y|
|R101MO7FBZP8OV|          4|           31|         31|   N|                Y|
|R101PYWOBNRA12|          5|           22|         22|   N|                Y|
|R1027NH9ETHTVA|          4|           49|         52|   N|                Y|
|R102X6PTY1J5FB|          5|           24|         30|   N|                Y|
|R1038G3ERTQXBB|          1|           21|         33|   N|     

In [14]:
# total unpaid reviews
unpaid_review_df.count()

109297

In [15]:
# total unpaid 5 star reviews
unpaid_review_df.filter("star_rating ==5").count()

49967

In [16]:
# percentage of unpaid 5 star reviews
unpaid_review_df.filter("star_rating ==5").count()/unpaid_review_df.count()*100

45.716716835777746